In [85]:
import warnings
warnings.simplefilter("ignore")

import datetime, os
from functools import partial
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow import keras
import keras_tuner as kt

from sklearn.metrics import mean_absolute_error

In [ ]:
# load the TensorBoard notebook extension
%load_ext tensorboard

In [87]:
# Set paths

DATA_PATH = 'Reservoir_Project/Data'
HP_TUNING_PATH = 'Reservoir_Project/Hyperparameter_Tuning'

In [88]:
basin_inflow_train = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_train.xlsx', index_col=0)
basin_inflow_validation = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_validation.xlsx', index_col=0)
basin_inflow_test = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_test.xlsx', index_col=0)

### Data windowing

In [89]:
# TensorFlow utility class for producing data windows from time series data

class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}
    
    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

In [90]:
"""
Window
- Given 60 days of history predict 30 days into the future. Why? A season is about 90 days in a CA WY
- Window size: 90
"""

window = WindowGenerator(input_width=60, label_width=1, shift=30,
                     train_df=basin_inflow_train, val_df=basin_inflow_validation, 
                     test_df=basin_inflow_test, label_columns=['INFLOW'])

window

In [91]:
# create a window of inputs and labels

def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # set shapes after slicing
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [92]:
# create a dataset of sliding windows over a time series dataframe

def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  # (input_window, label_window) pairs 
  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [93]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test

In [94]:
# each element is an (inputs, label) pair
window.train.element_spec

(TensorSpec(shape=(None, 60, 39), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))

In [95]:
# example batch
for inputs, labels in window.train.take(1):
  print(f'Inputs shape (batch, time, features): {inputs.shape}')
  print(f'Labels shape (batch, time, features): {labels.shape}')

Inputs shape (batch, time, features): (32, 60, 39)
Labels shape (batch, time, features): (32, 1, 1)


### Modeling

In [96]:
val_performance = {}
test_performance = {}

#### Create baseline

In [97]:
# tensorflow baseline utility class for data windowing

class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

In [98]:
baseline = Baseline(label_index=window.column_indices['INFLOW'])

baseline.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance['Baseline'] = baseline.evaluate(window.val)
test_performance['Baseline'] = baseline.evaluate(window.test, verbose=0)

30/30 [==============================] - 0s 481us/step - loss: 0.0870 - mean_absolute_error: 0.0870


#### LSTM

In [24]:
MAX_EPOCHS = 10

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                optimizer=tf.keras.optimizers.legacy.Adam())

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping, tensorboard_callback])
  return history

LSTM 1

In [25]:
"""
Inputs shape (batch, time, features): (32, 60, 39) - 32 batch size, 60 time steps, 39 features
"""

lstm_v1 = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 39]),
    tf.keras.layers.Dense(units=1)
])

In [26]:
history_lstm_v1 = compile_and_fit(lstm_v1, window)

Epoch 1/10
112/112 [==============================] - 2s 9ms/step - loss: 0.0771 - val_loss: 0.0756
Epoch 2/10
112/112 [==============================] - 1s 7ms/step - loss: 0.0613 - val_loss: 0.0769
Epoch 3/10
112/112 [==============================] - 1s 7ms/step - loss: 0.0560 - val_loss: 0.0822


In [27]:
# %tensorboard --logdir logs

In [28]:
val_performance['LSTM1'] = lstm_v1.evaluate(window.val)
test_performance['LSTM1'] = lstm_v1.evaluate(window.test, verbose=0)

print('LSTM 1 Validation performance: ', val_performance['LSTM1'])
print('LSTM 1 Test performance: ', test_performance['LSTM1'])

30/30 [==============================] - 0s 2ms/step - loss: 0.0822
LSTM 1 Validation performance:  0.08217944949865341
LSTM 1 Test performance:  0.038288358598947525


LSTM 2

In [29]:
lstm_v2 = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None, 39]),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(units=1)
])

In [30]:
history_lstm_v2 = compile_and_fit(lstm_v2, window)

Epoch 1/10
112/112 [==============================] - 4s 17ms/step - loss: 0.0618 - val_loss: 0.0704
Epoch 2/10
112/112 [==============================] - 2s 14ms/step - loss: 0.0418 - val_loss: 0.0638
Epoch 3/10
112/112 [==============================] - 2s 14ms/step - loss: 0.0367 - val_loss: 0.0646
Epoch 4/10
112/112 [==============================] - 2s 14ms/step - loss: 0.0320 - val_loss: 0.0706


In [31]:
val_performance['LSTM2'] = lstm_v2.evaluate(window.val)
test_performance['LSTM2'] = lstm_v2.evaluate(window.test, verbose=0)

print('LSTM 2 Validation performance: ', val_performance['LSTM2'])
print('LSTM 2 Test performance: ', test_performance['LSTM2'])

30/30 [==============================] - 0s 3ms/step - loss: 0.0706
LSTM 2 Validation performance:  0.0705932155251503
LSTM 2 Test performance:  0.05622085556387901


LSTM 3

In [32]:
RegularizedLSTM = partial(tf.keras.layers.LSTM,
                           dropout=0.01,
                           recurrent_dropout=0.01,
                           kernel_regularizer=tf.keras.regularizers.l2(0.05))

In [33]:
lstm_v3 = tf.keras.models.Sequential([
    RegularizedLSTM(32, return_sequences=True, input_shape=[None, 39]),
    RegularizedLSTM(32),
    tf.keras.layers.Dense(units=1)
])

In [34]:
history_lstm_v3 = compile_and_fit(lstm_v3, window)

Epoch 1/10
112/112 [==============================] - 7s 34ms/step - loss: 2.6534 - val_loss: 0.8579
Epoch 2/10
112/112 [==============================] - 4s 35ms/step - loss: 0.3875 - val_loss: 0.1369
Epoch 3/10
112/112 [==============================] - 4s 35ms/step - loss: 0.0946 - val_loss: 0.0694
Epoch 4/10
112/112 [==============================] - 4s 36ms/step - loss: 0.0680 - val_loss: 0.0657
Epoch 5/10
112/112 [==============================] - 4s 34ms/step - loss: 0.0655 - val_loss: 0.0685
Epoch 6/10
112/112 [==============================] - 4s 34ms/step - loss: 0.0653 - val_loss: 0.0688


In [35]:
val_performance['LSTM3'] = lstm_v3.evaluate(window.val)
test_performance['LSTM3'] = lstm_v3.evaluate(window.test, verbose=0)

print('LSTM 3 Validation performance: ', val_performance['LSTM3'])
print('LSTM 3 Test performance: ', test_performance['LSTM3'])

30/30 [==============================] - 0s 4ms/step - loss: 0.0688
LSTM 3 Validation performance:  0.0688057541847229
LSTM 3 Test performance:  0.040907714515924454


#### Hyperparameter tuning

In [36]:
def build_model(hp):
  lstm = keras.Sequential()

  ### Tuning hyperparameters ### 

  # Number of lstm layer units
  hp_units = hp.Int('units', min_value=32, max_value=128, step=32)

  # Dropout rate applied to input values
  hp_dropout = hp.Choice("dropout", [0.2, 0.3, 0.4, 0.5])

  # Recurrent dropout rate applied to hidden cell states between time steps
  hp_recurrent_dropout = hp.Choice("recurrent_dropout", [0.2, 0.3, 0.4, 0.5])
  
  # L2 regularization 
  hp_l2_reg = hp.Choice("l2", [0.001, 0.01, 0.02, 0.05])

  # Optimizer learning rate
  hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])

  ### Layers ### 
  
  lstm.add(tf.keras.layers.LSTM(units=hp_units, dropout=hp_dropout, recurrent_dropout=hp_recurrent_dropout, 
                  kernel_regularizer=tf.keras.regularizers.l2(l2=hp_l2_reg),
                  return_sequences=True, input_shape=[None, 39]))
    
  lstm.add(tf.keras.layers.LSTM(units=hp_units, dropout=hp_dropout, recurrent_dropout=hp_recurrent_dropout, 
                  kernel_regularizer=tf.keras.regularizers.l2(l2=hp_l2_reg)))
    
  lstm.add(keras.layers.Dense(1))    
    
  ### Compile ### 
           
  lstm.compile(loss=tf.keras.losses.MeanAbsoluteError(),
               optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=hp_learning_rate),
               metrics=['mean_absolute_error'])

  return lstm

In [37]:
tuner = kt.Hyperband(build_model,
                     objective='val_mean_absolute_error',
                     max_epochs=10,
                     factor=3,
                     directory=HP_TUNING_PATH,
                     project_name='reservoir_model_hp_tuning_v3')

Reloading Tuner from /Reservoir_Project/Hyperparameter_Tuning/reservoir_model_hp_tuning_v3/tuner0.json


In [38]:
MAX_EPOCHS = 10
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [39]:
# args for search are those used for model fit
tuner.search(window.train, epochs=MAX_EPOCHS, validation_data=window.val, callbacks=[early_stopping])

In [40]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete and below are the optimal values:

- Units: {best_hps.get('units')}
- Dropout: {best_hps.get('dropout')}
- Recurrent dropout: {best_hps.get('recurrent_dropout')}
- L2: {best_hps.get('l2')}
- Learning rate: {best_hps.get('learning_rate')}
""")


The hyperparameter search is complete and below are the optimal values:

- Units: 32
- Dropout: 0.4
- Recurrent dropout: 0.3
- L2: 0.02
- Learning rate: 0.01



#### Train

In [41]:
# first find the optimal number of training epochs

MAX_EPOCHS = 10

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=2,
                                                mode='min')

model = tuner.hypermodel.build(best_hps)
history = model.fit(window.train, epochs=MAX_EPOCHS, validation_data=window.val, callbacks=[early_stopping])

val_mae_per_epoch = history.history['val_mean_absolute_error']
best_epoch = val_mae_per_epoch.index(min(val_mae_per_epoch)) + 1 # find epoch of lowest validation MAE 
print('Best epoch: %d' % (best_epoch,))

Epoch 1/10
112/112 [==============================] - 6s 34ms/step - loss: 0.2247 - mean_absolute_error: 0.0740 - val_loss: 0.0616 - val_mean_absolute_error: 0.0532
Epoch 2/10
112/112 [==============================] - 4s 34ms/step - loss: 0.0746 - mean_absolute_error: 0.0680 - val_loss: 0.0705 - val_mean_absolute_error: 0.0636
Epoch 3/10
112/112 [==============================] - 4s 34ms/step - loss: 0.0720 - mean_absolute_error: 0.0660 - val_loss: 0.0630 - val_mean_absolute_error: 0.0576
Best epoch: 1


In [42]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=2,
                                                mode='min')
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

hypermodel = tuner.hypermodel.build(best_hps)
history = hypermodel.fit(window.train, epochs=best_epoch, validation_data=window.val, 
                         callbacks=[early_stopping, tensorboard_callback])

112/112 [==============================] - 6s 35ms/step - loss: 0.2198 - mean_absolute_error: 0.0729 - val_loss: 0.0692 - val_mean_absolute_error: 0.0647


In [43]:
val_performance['Hypermodel'] = hypermodel.evaluate(window.val)
test_performance['Hypermodel'] = hypermodel.evaluate(window.test, verbose=0)

print('Hypermodel Validation performance: ', val_performance['Hypermodel'])
print('Hypermodel Test performance: ', test_performance['Hypermodel'])

30/30 [==============================] - 0s 4ms/step - loss: 0.0692 - mean_absolute_error: 0.0647
Hypermodel Validation performance:  [0.0692351832985878, 0.06467919051647186]
Hypermodel Test performance:  [0.040652964264154434, 0.036096975207328796]


### Inference

In [44]:
windowTestBatch = window.test.take(1) # given 60 days of info, make prediction: batch of 32
windowTestBatch

<_TakeDataset element_spec=(TensorSpec(shape=(None, 60, 39), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))>

In [45]:
### Predicted Inflow ### 

predictedInflow = hypermodel.predict(windowTestBatch)

1/1 [==============================] - 0s 237ms/step


In [46]:
### Actual Inflow ### 

actualInflow = [] # predicting 30 days ahead: batch of 32

for inputs, labels in windowTestBatch.as_numpy_iterator():
  actualInflow = np.asarray(labels).flatten()

In [47]:
# compare predicted and actual inflow: MAE 

mean_absolute_error(actualInflow, predictedInflow)

0.036947846

In [83]:
# obtain absolute maximum inflow value for inverse scaling

basin_inflow = pd.read_excel(f'{DATA_PATH}/Custom/basin_inflow_no_dates.xlsx', index_col=0)
basin_inflow_test = basin_inflow[int(df_size * 0.9):] # last 10%
basin_inflow_test.reset_index(drop=True, inplace=True)
INFLOW_ABS_MAX = basin_inflow_test['INFLOW'].abs().max()

In [84]:
# inverse scaling for cfs prediction (predicting next 30 days)

def inverse_scaling(predictions):
    next_30_days = predictions[0:30]
    
    print('Next 30 Days of Inflow\n')
    for idx, target in enumerate(next_30_days):
        day = idx + 1
        print("{}. {:.3f} cfs".format(day, target[0] * INFLOW_ABS_MAX))

In [82]:
inverse_scaling(predictedInflow)

Next 30 Days of Inflow

1. 4202.729 cfs
2. 5746.163 cfs
3. 3263.437 cfs
4. 3719.474 cfs
5. 3237.862 cfs
6. 4481.116 cfs
7. 3136.901 cfs
8. 3682.255 cfs
9. 5172.745 cfs
10. 3076.100 cfs
11. 3712.135 cfs
12. 3342.944 cfs
13. 3180.339 cfs
14. 4432.857 cfs
15. 5544.335 cfs
16. 3896.251 cfs
17. 5737.286 cfs
18. 3015.898 cfs
19. 3763.401 cfs
20. 4742.780 cfs
21. 5941.011 cfs
22. 5164.979 cfs
23. 3095.047 cfs
24. 2320.619 cfs
25. 5168.629 cfs
26. 5729.252 cfs
27. 5968.246 cfs
28. 3952.693 cfs
29. 3548.037 cfs
30. 6362.657 cfs
